Import libraries

In [1]:
%%time

import os
import sagemaker
import boto3
import pickle
import pandas as pd
import numpy as np

from sagemaker import get_execution_role
region = boto3.Session().region_name
role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
CPU times: user 1.46 s, sys: 287 ms, total: 1.75 s
Wall time: 26.1 s


Create S3 bucket

In [5]:
# This creates a default S3 bucket where we will upload our model.
bucket = sagemaker.Session().default_bucket()

In [6]:
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

In [7]:
print(role)
print(region)
print(bucket)
print(bucket_path)

arn:aws:iam::533267128083:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
us-west-1
sagemaker-us-west-1-533267128083
https://s3-us-west-1.amazonaws.com/sagemaker-us-west-1-533267128083


Need to install the xgboost version

In [8]:
!conda install -y -c conda-forge xgboost

Retrieving notices: ...working... done
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::autopep8==2.0.4=pyhd8ed1ab_0
  - conda-forge/linux-64::black==24.2.0=py310hff52083_0
  - conda-forge/noarch::bleach==6.1.0=pyhd8ed1ab_0
  - conda-forge/noarch::plotly==5.19.0=pyhd8ed1ab_0
  - conda-forge/noarch::pytest==8.0.1=pyhd8ed1ab_1
  - conda-forge/noarch::qtpy==2.4.1=pyhd8ed1ab_0
  - conda-forge/linux-64::sip==6.7.12=py310hc6cd4ac_0
  - conda-forge/noarch::tqdm==4.66.2=pyhd8ed1ab_0
  - conda-forge/noarch::flask==3.0.2=pyhd8ed1ab_0
  - conda-forge/noarch::importlib_metadata==7.0.1=hd8ed1ab_0
  - conda-forge/noarch::nltk==3.8.1=pyhd8ed1ab_0
  - conda-forge/linux-64::pyqt5-sip==12.12.2=py310hc6cd4ac_5
  - conda-forge/noarch::pytoolconfig==1.2.5=pyhd8ed1ab_0
  - conda-forge/noarch::qdarkstyle==3.1=pyhd8ed1ab_0
  - conda-forge/noarch::qtawesome==1.3.0=pyh9208f05_0
  - cond

In [10]:
import xgboost
print (xgboost.__version__)

2.0.3


Load the model from pickle

In [11]:
with open ('model.pkl','rb') as file:
    model = pickle.load(file)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:16:30] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1712072639327/work/src/common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


Load the preprocessor

In [12]:
with open ('preprocessor.pkl','rb') as file:
    preprocessor = pickle.load(file)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipe

Read the test data and perform data pre-processing

In [13]:
df_test = pd.read_csv('test.csv')
test_sparse_arr = preprocessor.transform(df_test)
test_np_arr = test_sparse_arr.toarray()

Test prediction

In [14]:
prediction = model.predict(test_np_arr)

In [15]:
prediction

array([118074.14 , 151834.38 , 164716.77 , ..., 150627.88 , 119018.516,
       194844.45 ], dtype=float32)

Create a tar.gz model file as this is the format required by Sagemaker for deployment.

In [16]:
import tarfile

with tarfile.open('model.tar.gz', 'w:gz') as tar:
    tar.add('model.pkl')

Upload the pre-trained model to S3

In [17]:
prefix = "sagemaker/DEMO-xgboost-byo"
fObj = open("model.tar.gz","rb")
key = os.path.join(prefix,"model.tar.gz")
print (key)
boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(fObj)

sagemaker/DEMO-xgboost-byo/model.tar.gz


Set up Model Hosting

1. Create Sagemaker model
2. Create endpoint configuration

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
#### Get the built-in xgboost container image in Sagemaker to host our model

container = get_image_uri(boto3.Session().region_name,"xgboost")

In [25]:
from time import gmtime,strftime
model_name = "XGBHousingPricePrediction" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)
sm_client = boto3.client('sagemaker')
print (model_url)
primary_container = {
    
    "Image": container,
    "ModelDataUrl": model_url
}

sm_model = sm_client.create_model(ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container)
print (sm_model['ModelArn'])

https://s3-us-west-1.amazonaws.com/sagemaker-us-west-1-533267128083/sagemaker/DEMO-xgboost-byo/model.tar.gz
arn:aws:sagemaker:us-west-1:533267128083:model/XGBHousingPricePrediction2024-04-09-07-53-44


Create Endpoint Configuration

In [26]:
endpoint_config_name = "XGBHousingPricePrediction-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print (endpoint_config_name)

XGBHousingPricePrediction-2024-04-09-07-59-42


In [29]:
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants = [
        {
            "InstanceType": "ml.t2.medium",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
        
        
    ],

)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Endpoint Config Arn: arn:aws:sagemaker:us-west-1:533267128083:endpoint-config/XGBHousingPricePrediction-2024-04-09-07-59-42


Create End point

In [ ]:
import time
endpoint_name = "XGBHousingPricePredictionEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print (endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

XGBHousingPricePredictionEndpoint-2024-04-09-08-08-08
arn:aws:sagemaker:us-west-1:533267128083:endpoint/XGBHousingPricePredictionEndpoint-2024-04-09-08-08-08
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating


Launching necessary AWS Services for End-to-End Communication using API gateway


1. Create a Lambda function with the below mentioned python code, that calls the SageMaker runtime invoke_endpoint and returns the prediction

2. Create a REST API (API Gateway) and integrate with the Lambda function

3. Testing the final Deployment from local client